<a href="https://colab.research.google.com/github/grendelaglaeca/NMA-Computational-Neuroscience/blob/main/Copy_of_Group_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest as smt
import seaborn as sns
import networkx as nx
from networkx.algorithms import approximation as approx
from google.colab import files
import statistics as stats
import pprint as pp
import itertools
np.set_printoptions(threshold=np.inf)
pd.set_option("display.max_rows", None, "display.max_columns", None)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Necessary for visualization
!pip install nilearn --quiet
from nilearn import plotting, datasets

     |████████████████████████████████| 10.0 MB 3.7 MB/s 


/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [ ]:
#@title Figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

# Basic parameters

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

# You may want to limit the subjects used during code development.
# This will use all subjects:
subjects = range(N_SUBJECTS)

# Downloading data

The rest and task data are shared in different files, but they will unpack into the same directory structure.

Each file is fairly large and will take some time to download. If you are focusing only on rest or task analyses, you may not want to download only that dataset.

We also separately provide some potentially useful behavioral covariate information.

In [ ]:
fname = "hcp_rest.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/bqp7m/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

In [ ]:
fname = "hcp_task.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/s4h8j/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

In [ ]:
fname = "hcp_covariates.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/x5p4g/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

In [ ]:
fname = f"{HCP_DIR}/atlas.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/j5kuc/download

## Loading region information

Downloading either dataset will create the `regions.npy` file, which contains the region name and network assignment for each parcel.

Detailed information about the name used for each region is provided [in the Supplement](https://static-content.springer.com/esm/art%3A10.1038%2Fnature18933/MediaObjects/41586_2016_BFnature18933_MOESM330_ESM.pdf) to [Glasser et al. 2016](https://www.nature.com/articles/nature18933).

Information about the network parcellation is provided in [Ji et al, 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6289683/).

In [ ]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    myelin=regions[2].astype(np.float),
)

We also provide the [parcellation on the fsaverage5 surface](https://figshare.com/articles/HCP-MMP1_0_projected_on_fsaverage/3498446) and approximate MNI coordinates of each region, which can be useful for visualization:

In [ ]:
with np.load(f"{HCP_DIR}/atlas.npz") as dobj:
  atlas = dict(**dobj)

# Helper functions


## Data loading

In [ ]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
  ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids

def load_timeseries(subject, name, runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.
  
  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment 
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
      load_single_timeseries(subject, offset + run, remove_mean) for run in runs
  ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


def load_single_timeseries(subject, bold_run, remove_mean=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

def load_evs(subject, name, condition):
  """Load EV (explanatory variable) data for one task condition.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of task
    condition (str) : Name of condition

  Returns
    evs (list of dicts): A dictionary with the onset, duration, and amplitude
      of the condition for each run.

  """
  evs = []
  for id in get_image_ids(name):
    task_key = BOLD_NAMES[id - 1]
    ev_file = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{condition}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev = dict(zip(["onset", "duration", "amplitude"], ev_array))
    evs.append(ev)
  return evs

## Task-based analysis

In [ ]:
def condition_frames(run_evs, skip=0):
  """Identify timepoints corresponding to a given condition in each run.

  Args:
    run_evs (list of dicts) : Onset and duration of the event, per run
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    frames_list (list of 1D arrays): Flat arrays of frame indices, per run

  """
  frames_list = []
  for ev in run_evs:

    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)

    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)

    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(skip, d) for s, d in zip(start, duration)]

    frames_list.append(np.concatenate(frames))

  return frames_list


def selective_average(timeseries_data, ev, skip=0):
  """Take the temporal mean across frames for a given condition.

  Args:
    timeseries_data (array or list of arrays): n_parcel x n_tp arrays
    ev (dict or list of dicts): Condition timing information
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    avg_data (1D array): Data averagted across selected image frames based
    on condition timing

  """
  # Ensure that we have lists of the same length
  if not isinstance(timeseries_data, list):
    timeseries_data = [timeseries_data]
  if not isinstance(ev, list):
    ev = [ev]
  if len(timeseries_data) != len(ev):
    raise ValueError("Length of `timeseries_data` and `ev` must match.")

  # Identify the indices of relevant frames
  frames = condition_frames(ev, skip)

  # Select the frames from each image
  selected_data = []
  for run_data, run_frames in zip(timeseries_data, frames):
    run_frames = run_frames[run_frames < run_data.shape[1]]
    selected_data.append(run_data[:, run_frames])

  # Take the average in each parcel
  avg_data = np.concatenate(selected_data, axis=-1).mean(axis=-1)

  return avg_data



```
# This is formatted as code
```

# Derive Connectome

In [ ]:
from nilearn.connectome import sym_matrix_to_vec

#Indices of brain networks of interest
network_idx = np.where((region_info['network'] == 'Language'))# | (region_info['network'] == 'Cingulo-Oper'))[0]

#Returns ROIxROI connectome 
arrays = []

for i in range(2):
  time_series = load_timeseries(subject=i, name="rest", runs=1)
  corr = np.corrcoef(time_series)
  net_corr = corr[network_idx]
  net_corr2 = net_corr.T[network_idx]
  flat_corr = sym_matrix_to_vec(net_corr2, discard_diagonal=True)
  arrays.append(flat_corr)

#Dataframe of ROI correlation
corr_df = pd.DataFrame(np.stack(arrays))

#Returns whole brain flattened connectome
whole_brain_arrays = []

for i in range(1):
  time_series = load_timeseries(subject=i, name="rest", runs=1)
  whole_brain_corr = np.corrcoef(time_series)
  whole_brain_flat_corr = sym_matrix_to_vec(whole_brain_corr, discard_diagonal=True)
  whole_brain_arrays.append(whole_brain_flat_corr)

#Dataframe of whole brain correlation
whole_brain_corr_df = pd.DataFrame(np.stack(whole_brain_arrays))

# **Network functions** *(bold chunks are the ones we developed and the main focus of the project)*

In [ ]:
time_series_network = select_network('Language', 'Language', 1)
windows = sliding_window(time_series_network, 30, 30)

NameError: ignored

In [ ]:
def select_network(network, task, subject):
  """Gets data from regions belonging to the specified network from the specified task.
  """
  # Select network regions.
  time_series = load_timeseries(subject=subject, name=task) # Borrowed function.
  network_indices = np.where((region_info['network'] == network)) # Requires region_info.
  network_indices_list = list(network_indices[0])
  time_series_network = time_series[network_indices_list,:]
  # Select task moments.
  ev = load_evs(subject, task, "story") ###################################################### SPECIFY THE TASK WITHIN THE RUN ("math", "story")
  frames_list = condition_frames(ev, subject)
  frames_list_flat = list(itertools.chain(*frames_list))
  time_series_network = time_series_network[:, frames_list_flat]
  return time_series_network


def iter_corr_negative_only(time_series_network, threshold, binary):
  """Single dataset Spearman correlate region-wise, p-value filter and FDR correct.

  Args:
    time_series_network (array): n_parcel x n_tp arrays.
    threshold (float): p-value significance level.
    binary (bool): replace significant correlation coefficients with 1.

  Returns:
    correlation_matrix_corrected (dataframe): correlation matrix of regions x regions.
  """
  # Correlate regions one by one (significance-sensitive).
  correlation_matrix = pd.DataFrame(index=range(len(time_series_network)), columns=range(len(time_series_network)))
  p_values = []
  # nonsignificant_count = 0
  for row1 in range(len(time_series_network)):
    for row2 in range(row1 + 1, len(time_series_network)):
      if all(v == 0 for v in time_series_network[row1]) or all(v == 0 for v in time_series_network[row2]):
        correlation_matrix.at[row1, row2] = 0
        p_values.append(1)
      if all(v == 0 for v in time_series_network[row1]) == False and all(v == 0 for v in time_series_network[row2]) == False:
        r, p = scipy.stats.spearmanr(time_series_network[row1], time_series_network[row2])
        if p < threshold and r < 0:
          if binary == True:
            correlation_matrix.at[row1, row2] = 1
          if binary == False:
            correlation_matrix.at[row1, row2] = r
        if p >= threshold or r >= 0:
          correlation_matrix.at[row1, row2] = 0
        # nonsignificant_count += 1
        p_values.append(p)
  correlation_matrix = correlation_matrix.astype("float64")
  # print(nonsignificant_count)
 
  # FDR correction.
  correlation_matrix_corrected = correlation_matrix.copy()
  reject, pvals_corrected = smt.fdrcorrection(p_values, alpha=0.05, method='indep', is_sorted=False)
  p_values_corrected = pd.DataFrame(index=range(len(time_series_network)), columns=range(len(time_series_network)))
  # fdr_filtered_count = 0
  item1 = 0
  for row1 in range(len(time_series_network)):
    for row2 in range(row1 + 1, len(time_series_network)):
      if pvals_corrected[item1] >= threshold:
        correlation_matrix_corrected.at[row1, row2] = 0
        # fdr_filtered_count += 1
      item1 += 1
  # print(fdr_filtered_count-nonsignificant_count)

  return correlation_matrix_corrected


def iter_corr_positive_only(time_series_network, threshold, binary):
  """Single dataset Spearman correlate region-wise, p-value filter and FDR correct.

  Args:
    time_series_network (array): n_parcel x n_tp arrays.
    threshold (float): p-value significance level.
    binary (bool): replace significant correlation coefficients with 1.

  Returns:
    correlation_matrix_corrected (dataframe): correlation matrix of regions x regions.
  """
  # Correlate regions one by one (significance-sensitive).
  correlation_matrix = pd.DataFrame(index=range(len(time_series_network)), columns=range(len(time_series_network)))
  p_values = []
  # nonsignificant_count = 0
  for row1 in range(len(time_series_network)):
    for row2 in range(row1 + 1, len(time_series_network)):
      if all(v == 0 for v in time_series_network[row1]) or all(v == 0 for v in time_series_network[row2]):
        correlation_matrix.at[row1, row2] = 0
        p_values.append(1)
      if all(v == 0 for v in time_series_network[row1]) == False and all(v == 0 for v in time_series_network[row2]) == False:
        r, p = scipy.stats.spearmanr(time_series_network[row1], time_series_network[row2])
        if p < threshold and r > 0:
          if binary == True:
            correlation_matrix.at[row1, row2] = 1
          if binary == False:
            correlation_matrix.at[row1, row2] = r
        if p >= threshold or r <= 0:
          correlation_matrix.at[row1, row2] = 0
        # nonsignificant_count += 1
        p_values.append(p)
  correlation_matrix = correlation_matrix.astype("float64")
  # print(nonsignificant_count)
 
  # FDR correction.
  correlation_matrix_corrected = correlation_matrix.copy()
  reject, pvals_corrected = smt.fdrcorrection(p_values, alpha=0.05, method='indep', is_sorted=False)
  p_values_corrected = pd.DataFrame(index=range(len(time_series_network)), columns=range(len(time_series_network)))
  # fdr_filtered_count = 0
  item1 = 0
  for row1 in range(len(time_series_network)):
    for row2 in range(row1 + 1, len(time_series_network)):
      if pvals_corrected[item1] >= threshold:
        correlation_matrix_corrected.at[row1, row2] = 0
        # fdr_filtered_count += 1
      item1 += 1
  # print(fdr_filtered_count-nonsignificant_count)

  return correlation_matrix_corrected


def iter_corr(time_series_network, threshold, binary):
  """Single dataset Spearman correlate region-wise, p-value filter and FDR correct.

  Args:
    time_series_network (array): n_parcel x n_tp arrays.
    threshold (float): p-value significance level.
    binary (bool): replace significant correlation coefficients with 1.

  Returns:
    correlation_matrix_corrected (dataframe): correlation matrix of regions x regions.
  """
  # Correlate regions one by one (significance-sensitive).
  correlation_matrix = pd.DataFrame(index=range(len(time_series_network)), columns=range(len(time_series_network)))
  p_values = []
  # nonsignificant_count = 0
  for row1 in range(len(time_series_network)):
    for row2 in range(row1 + 1, len(time_series_network)):
      if all(v == 0 for v in time_series_network[row1]) or all(v == 0 for v in time_series_network[row2]):
        correlation_matrix.at[row1, row2] = 0
        p_values.append(1)
      if all(v == 0 for v in time_series_network[row1]) == False and all(v == 0 for v in time_series_network[row2]) == False:
        r, p = scipy.stats.spearmanr(time_series_network[row1], time_series_network[row2])
        if p < threshold:
          if binary == True:
            correlation_matrix.at[row1, row2] = 1
          if binary == False:
            correlation_matrix.at[row1, row2] = r
        if p >= threshold:
          correlation_matrix.at[row1, row2] = 0
        # nonsignificant_count += 1
        p_values.append(p)
  correlation_matrix = correlation_matrix.astype("float64")
  # print(nonsignificant_count)
 
  # FDR correction.
  correlation_matrix_corrected = correlation_matrix.copy()
  reject, pvals_corrected = smt.fdrcorrection(p_values, alpha=0.05, method='indep', is_sorted=False)
  p_values_corrected = pd.DataFrame(index=range(len(time_series_network)), columns=range(len(time_series_network)))
  # fdr_filtered_count = 0
  item1 = 0
  for row1 in range(len(time_series_network)):
    for row2 in range(row1 + 1, len(time_series_network)):
      if pvals_corrected[item1] >= threshold:
        correlation_matrix_corrected.at[row1, row2] = 0
        # fdr_filtered_count += 1
      item1 += 1
  # print(fdr_filtered_count-nonsignificant_count)

  return correlation_matrix_corrected


def sliding_window(time_series_network, window_size, step):
  """Chop an array (time series) into equally-sized subsets of contiguous
     columns (windows) for every step.

  Args:
    time_series_network (array): n_parcel x n_tp arrays.
    winsow_size (int.): amount of columns (time steps) of the window.
    step (int.): step size for window placement.

  Returns:
    windows (list): ordered list of subsets (arrays) of contiguous columns
    (windows) of an array (time series) of window_size size.
  """
  windows = []
  for i in range(0, (len(time_series_network[0]) - window_size)+1, step):
    window = time_series_network[:, range(i, i + window_size)]
    windows.append(window)

  return windows


def df_to_graph(corr_matrices):
  """Documentation pending but basically takes a list of correlation matrices
  and turns each of them into a graph object through a number of bureaucratic
  data type transformations, this function is hideous and I hate it. Pandas is
  so much better than Numpy but NetworkX apparently thinks otherwise. End of rant.
  """
  graphs = []
  matrices = []
  dataframes = []
  for elem in corr_matrices:
    array = elem.to_numpy()
    np.nan_to_num(array, copy=False, nan=0.0, posinf=None, neginf=None)
    new_array = array + array.T
    for i in range(len(new_array)):
     new_array[i][i] = 0
    matrices.append(new_array)
    graph = nx.Graph(new_array)
    graphs.append(graph)
    dataframe = pd.DataFrame(new_array)
    dataframes.append(dataframe)

  return graphs, matrices, dataframes


def get_graphs(network, task, subject, threshold, window_size, step, binary):
  """Documentation pending but basically iters all previous functions, taking in
  a time series and some basic parameters (see above) and returns a list of graphs.
  """
  time_series_network = select_network(network, task, subject)
  windows = sliding_window(time_series_network, window_size, step)
  corr_matrices = []
  for i in range(len(windows)):
    correlation_matrix_corrected = iter_corr_positive_only(windows[i], threshold, binary)
    corr_matrices.append(correlation_matrix_corrected)
  graphs, matrices, dataframes = df_to_graph(corr_matrices)
    
  return graphs, matrices, dataframes

# **Analysis functions**

In [ ]:
windows[0][0][0]

In [ ]:
len(windows[0])

In [ ]:
average_activity

In [ ]:
np.mean(windows[1])

In [ ]:
(windows[0].shape[0] * windows[0].shape[1])

In [ ]:
np.sum(windows[0]) / (windows[0].shape[0] * windows[0].shape[1])

In [ ]:
  # time_series_network = select_network(network, task, subject)
  # windows = sliding_window(time_series_network, window_size, step)
  average_activity = []
  for window in windows:
    sum = 0
    for axis0 in range(len(window)):
      for axis1 in range(len(window[0])):
        sum =+ window[axis0][axis1]
    mean = sum/(len(window)**2)
    average_activity.append(mean)

In [ ]:
def coincidence_ratio(dataframes):
  """Computes coincidence ratio (matching values/total values) between correlation
  matrices. Naturally it performs poorly with weighted matrices.
  """
  coincidences = pd.DataFrame(index=range(len(dataframes)), columns=range(len(dataframes)))
  for dataframe1 in range(len(dataframes)):
    for dataframe2 in range(dataframe1 + 1, len(dataframes)):
      count = 0
      total = 0
      for region1 in range(len(dataframes[dataframe1])):
        for region2 in range(region1 + 1, len(dataframes[dataframe1])):
          if dataframes[dataframe1].at[region1, region2] == dataframes[dataframe2].at[region1, region2]:
            count += 1
            total += 1
          if dataframes[dataframe1].at[region1, region2] != dataframes[dataframe2].at[region1, region2]:
            total +=1
      ratio = count / total
      coincidences.at[dataframe1, dataframe2] = ratio

  return coincidences


def mean_signal(network, task, subject, window_size, step):
  """Averages all values in a window, for all windows.
  """
  time_series_network = select_network(network, task, subject)
  windows = sliding_window(time_series_network, window_size, step)
  average_activity = []
  for window in windows:
    sum = 0
    for axis0 in range(len(window)):
      for axis1 in range(len(window[0])):
        sum =+ window[axis0][axis1]
    mean = sum/(len(window) * window_size)
    average_activity.append(mean)

  return average_activity


def stats_trio(name, list_temp, parameters_names, parameters_list):
  """Computes the mean, standard deviation and coefficient of variation of the given parameters.
  """
  parameters_names.append(name + "_mean")
  #parameters_names.append(name + "_standard_deviation")
  parameters_names.append(name + "_coefficient_of_variation")
  if stats.mean(list_temp) != 0 and stats.stdev(list_temp) != 0:
    parameters_list.append(stats.mean(list_temp))
    #parameters_list.append(stats.stdev(list_temp))
    parameters_list.append(stats.stdev(list_temp)/stats.mean(list_temp))
  if stats.mean(list_temp) == 0 and stats.stdev(list_temp) != 0:
    parameters_list.append(0)
    #parameters_list.append(stats.stdev(list_temp))
    parameters_list.append(stats.stdev(list_temp))
  if stats.mean(list_temp) == 0 and stats.stdev(list_temp) == 0:
    parameters_list.append(0)
    #parameters_list.append(0)
    parameters_list.append(0)
  if stats.mean(list_temp) != 0 and stats.stdev(list_temp) == 0:
    parameters_list.append(stats.mean(list_temp))
    #parameters_list.append(0)
    parameters_list.append(0)


def graph_analysis(graph):
  """Computes a series of graph theoretical parameters (and creates a lable list).
  """
  parameters_names = []
  parameters_list = []
  parameters_names.append("Average_clustering_coefficient") # Average of: fraction of existing triangles of a node over all possibles, or, fraction of existing edges between adjacent nodes of a node over all possibles.
  parameters_list.append(approx.average_clustering(graph))
  #parameters_names.append("Minimum_maximal_matching_size") # Number of nodes in the smallest (minimum) matching (groupd of non-adjacent nodes (no shared immediate neighbours)) including all nodes it possibly can (maximal).
  #parameters_list.append(len(approx.min_maximal_matching(graph)))
  ##parameters_names.append("Degree_assortativity_coefficient")
  ##parameters_list.append(nx.degree_assortativity_coefficient(graph))
  list_temp = list(dict.values(nx.average_neighbor_degree(graph, source = "out", target = "in"))) # FIXME what are source and target attributes? They impact on results.
  stats_trio("Average_neighbor_degree", list_temp, parameters_names, parameters_list)
  ##parameters_names.append("Minimum_edge_cover") # Minimum number of the edges of a graph needed so that every node has at least one edge, meant for bipartite graphs but I believe it doesn't matter though.
  ##parameters_list.append(len(nx.min_edge_cover(graph)))
  #list_temp = list(dict.values(nx.degree_centrality(graph)))
  #stats_trio("Degree_centrality", list_temp, parameters_names, parameters_list)
  ##list_temp = list(dict.values(nx.eigenvector_centrality(graph)))
  ##stats_trio("Eigenvector_centrality", list_temp, parameters_names, parameters_list) # Measures connectivity (degree?) of neighbors.
  ##list_temp = list(dict.values(nx.katz_centrality(graph)))
  ##stats_trio("Katz_centrality", list_temp, parameters_names, parameters_list) # Measures distance to all reachable nodes with distance attenuation (?).
  list_temp = list(dict.values(nx.closeness_centrality(graph)))
  stats_trio("Closeness_centrality", list_temp, parameters_names, parameters_list) # Average of: shortest path between a pair of nodes.
  list_temp = list(dict.values(nx.betweenness_centrality(graph)))
  stats_trio("Betweenness_centrality", list_temp, parameters_names, parameters_list) # Averag of: ratio of shortests paths that use a node.
  ##list_temp = list(dict.values(nx.edge_betweenness_centrality(graph)))
  ##stats_trio("Edge_betweenness_centrality", list_temp, parameters_names, parameters_list) # Averag of: ratio of shortests paths that use an edge (confirm).
  list_temp = []
  for i in nx.degree(graph):
	  list_temp.append(i[1])
  stats_trio("Degree", list_temp, parameters_names, parameters_list) # Degree.
  parameters_names.append("Density") # Fraction of existing edges over all possible edges.
  parameters_list.append(nx.density(graph))
  #list_temp = [] 
  #for i in list(nx.nodes(graph)):
	#  list_temp.append(len((list(nx.non_neighbors(graph, i)))))
  #stats_trio("Non-neighbors", list_temp, parameters_names, parameters_list)
  #parameters_names.append("Edge_count")
  #parameters_list.append(nx.number_of_edges(graph))
  ##parameters_names.append("Small_world_coefficient")
  ##parameters_list.append(nx.algorithms.smallworld.sigma(graph))

  return parameters_names, parameters_list


def iter_graph_analysis(graphs, network, task, subject, window_size, step, threshold2, binary2):
  """Takes the graph_analysis functions and performs it over all graphs in an
  ordered way, then correlates all parameter timeseries between themselves and
  and returns the corresponding matrix. I am ashamed of how unelegant this
  function is (it was past midnight), I apologize for it and promise to fix it.
  """
  parameters_names, parameters_list = graph_analysis(graphs[0])
  for elem in parameters_names:
    vars()[str(elem + "_timeseries")] = []
  for graph in graphs:
    parameters_names, parameters_list = graph_analysis(graph)
    for i in range(len(parameters_list)):
      vars()[str(parameters_names[i] + "_timeseries")].append(parameters_list[i])
  all_parameters_array = []
  for elem in parameters_names:
    all_parameters_array.append(vars()[str(elem + "_timeseries")])
  average_activation = mean_signal(network, task, subject, window_size, step)
  all_parameters_array.append(average_activation)
  parameters_names.append("Average_activation_timeseries")
  parameters_corr_array = iter_corr_negative_only(all_parameters_array, threshold2, binary2)
  parameters_corr = pd.DataFrame(parameters_corr_array)
  parameters_corr.index = parameters_names
  parameters_corr.columns = parameters_names

  return parameters_corr, parameters_names, all_parameters_array

# Ok last function, I promise. This just puts everything together but it still deals with a single subject.
def activity_parameters_corr(network, task, subject, threshold, threshold2, window_size, step, binary, binary2):

  # Call task data from network regions and create dynamic graphs.
  graphs, matrices, dataframes = get_graphs(network, task, subject, threshold, window_size, step, binary)

  # Compute dynamic average activation.
  average_activity = mean_signal(network, task, subject, window_size, step)

  # Compute dynamic graph theoretical parameters.
  parameters_corr, parameters_names, all_parameters_array = iter_graph_analysis(graphs, network, task, subject, window_size, step, threshold2, binary2)

  # Compute dynamic binary similarity between graphs. I guess we won't be using this. Might be usefull for clustering...? (@Matt)
  coincidences = coincidence_ratio(dataframes)

  return parameters_corr, graphs, all_parameters_array, parameters_names


# **Iterative functions**

In [ ]:
# This bit was super improvised and is very disorganized so if it seems random, redundant or difficult to comprehend, it's my fault.

# Now we grab what we defined before and loop it over all subjects (or some of them).
def iter_over_subjects(network, subjects, task, threshold, threshold2, window_size, step, binary, binary2):
  parameters_corr_all_subjects = pd.DataFrame()
  CVdf_crop_all_subjects = pd.DataFrame()
  for subject in range(subjects): # if you want to do all subjects: " for subjects in range(len(N_SUBJECTS)): "
    try:
      # The goal of this small chunk is to store in one array the correlations between each graph metric and average activity only, for every subject.
      # We run the full analysis on one subject at a time, we save parameters_corr (a correlation matrix between ALL metrics), graphs (the networks (one per window)) and all_parameters_array (a table of all metrics (23 I believe) across time (so one value per window, per metric)).
      parameters_corr, graphs, all_parameters_array, parameters_names = activity_parameters_corr(network, task, subject, threshold, threshold2, window_size, step, binary, binary2)
      # parameters_corr_all_subjects will keep track of correlations between all metrics and average activation only (of all subjects), for this we grab a subset of the main correlation matrix with [:-1] (the las column).
      parameters_corr_all_subjects[subject] = parameters_corr["Average_activation_timeseries"][:-1]
      # Not important: since it takes some time to compute, the function lets you know when each subject is ready.
      print(str("Done with subject nº " + str(subject)))
      
      # The goal behind this chunk is to evaluate how much graph metrics evolve over time (with the standard deviation normalized by the mean ("coefficient of variation" or "CV")).
      # Once again I'm sorry for the ugly code, here we turn the array with all the parameters of a subject into a dataframe just because I prefer dataframes (now is when I get lazy with variable names).
      CVdf = pd.DataFrame(all_parameters_array)
      # We give names to the rows of the dataframe so we know what metrics we're looking at.
      CVdf.index = parameters_names
      # We add a couple extra columns storing some basic statistics of every metric across time.
      CVdf["mean"] = CVdf.mean(axis=1) # Average metrics along the time series.
      CVdf["SD"] = CVdf.std(axis=1) # You get the idea.
      CVdf["CV"] = abs(CVdf.std(axis=1)/CVdf.mean(axis=1)) # This is the one we care about, how each metric varies across time, scaled for magnitude.
      # We pick this last column and get rid of the rest to declutter (specially since we will be appending for all subjects).
      CVdf_crop = CVdf["CV"]
      # We store this data of each subject in a big dataframe of all subjects.
      CVdf_crop_all_subjects[subject] = CVdf_crop
    except:
      print(subject)

  return parameters_corr_all_subjects, CVdf_crop_all_subjects

# Ok so to wrap things up (again sorry for the mess): we ended up with two beautiful dataframes: parameters_corr_all_subjects and CVdf_crop_all_subjects.
# These dataframes follow the same logic in that both list graph metrics as rows and subjects as columns, the difference between the two is that
# parameters_corr_all_subjects stores the CORRELATION of each metric with the average activity, and CVdf_crop_all_subjects stores the COEFFICIENT OF VARIATION
# of each metric across time.

# This becomes much clearer once you see the dataframes (visualization is better if we turn them into strings, for some reason), hence the following lines.
# Remember that columns are subjects (which we still need to summarize because we can't interpret 339 results at once (e.g., sum, average, etc.)) and rows are graph metrics.

def get_results(parameters_corr_all_subjects, CVdf_crop_all_subjects):
  parameters_corr_all_subjects_sum = parameters_corr_all_subjects.sum(axis=1)
  #parameters_corr_all_subjects_sum.plot.barh()
  CVdf_crop_all_subjects_mean = CVdf_crop_all_subjects.mean(axis=1)
  CVdf_crop_all_subjects_mean_crop = CVdf_crop_all_subjects_mean.copy()
  CVdf_crop_all_subjects_mean_crop = CVdf_crop_all_subjects_mean_crop.drop("Average_activation_timeseries", 0)
  #CVdf_crop_all_subjects_mean_crop.plot.barh()

  return parameters_corr_all_subjects_sum, CVdf_crop_all_subjects_mean_crop

# **Get the graphs**

In [ ]:
# Here we (finally) permorm the actual analysis, for this we have to specify the following parameters:
network =      "Language"
task =         "Language"
subjects =     20
threshold =    0.05 # of the between-region correlations for creating the correlation matrix.
threshold2 =   0.05 # of the average activity-graph metric correaltions.
window_size =  30 # Minimum according to Apoorva.
step =         30 # Apoorva also suggested non-overlapping to avoid intrinsic correlation.
binary =       True # of the between-region correlations for creating the correlation matrix.
binary2 =      True # of the average activity-graph metric correaltions.

parameters_corr_all_subjects, CVdf_crop_all_subjects = iter_over_subjects(network, subjects, task, threshold, threshold2, window_size, step, binary, binary2)
parameters_corr_all_subjects_sum, CVdf_crop_all_subjects_mean_crop = get_results(parameters_corr_all_subjects, CVdf_crop_all_subjects)


In [ ]:
print(parameters_corr_all_subjects_sum)
parameters_corr_all_subjects_sum = np.flip(parameters_corr_all_subjects_sum)
parameters_corr_all_subjects_sum.plot.barh()

In [ ]:
print(CVdf_crop_all_subjects_mean_crop)
CVdf_crop_all_subjects_mean_crop = np.flip(CVdf_crop_all_subjects_mean_crop)
CVdf_crop_all_subjects_mean_crop.plot.barh()

# Other stuff (ignore)

In [ ]:
#CVdf_crop_all_subjects_mean_crop.plot.barh()

s1 = parameters_corr_all_subjects.to_string()
s2 = CVdf_crop_all_subjects.to_string()
print(" ")
print("Correlations between each metric and average activation for each subject:")
print(s1)
print(" ")
print("Coefficient of varation of each metric across time for each subject:")
print(s2)

# Distribution 

sns.displot(listY, kind="kde")

# Joint plot for selected metric correlations.

listX = CVdf_crop_all_subjects.loc["Average_activation_timeseries"]
listY = CVdf_crop_all_subjects.loc["Average_clustering_coefficient"]
sns.jointplot(listX, listY, kind="reg")

In [ ]:
# parameters_corr_all_subjects
# CVdf_crop_all_subjects
  
CVdf_crop_all_subjects["mean"] = CVdf_crop_all_subjects.mean(axis=1)
CVdf_crop_all_subjects["SD"] = CVdf_crop_all_subjects.std(axis=1)
CVdf_crop_all_subjects["CV"] = abs(CVdf_crop_all_subjects.std(axis=1)/CVdf_crop_all_subjects.mean(axis=1))

print(CVdf_crop_all_subjects["CV"])
print(parameters_corr_all_subjects)

parameters_corr_all_subjects["sum"] = parameters_corr_all_subjects.sum(axis=1)
print(parameters_corr_all_subjects["sum"])
sns.barplot(parameters_corr_all_subjects["sum"])

# Get activity-metric correlation frecuency across all subjects.

parameters_corr_all_subjects["sum"].plot.barh()

# **Plots**

In [ ]:
# The most extra part of the script.
# Don't plot both figures at once (or do so and see what happens). 

nx.draw(graphs[8]) # There are several specific plotting algorithms (circular, force, etc) that you can learn more about here: https://networkx.org/documentation/stable/reference/drawing.html?highlight=draw
heatmap = sns.heatmap(parameters_corr, xticklabels=True, yticklabels=True) # The long labels ruin everything.

# Other stuff that came with the notebook

In [ ]:
#Plot connectivity matrix
import numpy as np
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure

#Generate pseudo labels for now
labels = []
for i in range(360):
  labels.append('label' + str(i))

# Mask the main diagonal for visualization:
np.fill_diagonal(whole_brain_corr, 0)

# Plot corr matrix
plotting.plot_matrix(correlation_matrix, figure=(12, 10), labels=labels,
                     vmax=0.8, vmin=-0.8, reorder=True)